# Lesson 5: Predefined Markov Chains

In [1]:
import marmote.core as marmotecore
import marmote.markovchain as mmc
import numpy as np

Some of the well-known Markov models are implemented in `Marmote` as objects heritating from `MarkovChain`. Their current list is:

* `TwoStateDiscrete`
* `TwoStateContinuous`
* `Homogeneous1DRandomWalk`
* `Homogeneous1DBirthDeath`
  * `PoissonProcess`
* `HomogeneousMultiDRandomWalk`
* `HomogeneousMultiDBirthDeath`
* `MMPP` 

## The continuous-time birth-death process

### Infinite-state birth-death process

In [2]:
mc = mmc.Homogeneous1DBirthDeath( 0.5, 1.0 )

Although it has theoretically an infinite state space, this chain can be simulated.

Simulating the chain with options:

* 10.0 units of time
* no stats collected
* no trajectory kept in memory
* trajectory printed along the way
* increments in times displayed as well

In [3]:
simres = mc.SimulateChain( 10.0, stats=False, traj=False, _print=True, withIncrements=True )

[   0]     0.000000        0   0.000000
[   1]     3.373912        1   3.373912
[   2]     3.421189        0   0.047277
[   3]     4.869436        1   1.448248
[   4]     5.627942        2   0.758506
[   5]     5.866810        3   0.238867
[   6]     6.348972        2   0.482162
[   7]     6.482716        1   0.133744
[   8]     6.946735        0   0.464019
[   9]     7.604426        1   0.657691
[  10]     8.506338        0   0.901911
[  11]    10.000000        1   1.493662


Computing the stationary distribution. It is well-known that this distribution is geometric.

In [4]:
stadis = mc.StationaryDistribution()

In [5]:
print(stadis)
print(stadis.Mean())
print(stadis.getProba(0))
print(stadis.getProba(1))
print(stadis.Ccdf(4))

Geometric on N with proba      0.5: P(k) = 0.5 x (0.5)^k, k=0..+oo
1.0
0.5
0.25
0.03125


Embedding and Uniformizing.

Uniformizing results in another well-known Markov chain.

In [6]:
umc = mc.Uniformize()
print(umc.className())

Homogeneous1DRandomWalk


However embedding does not result in a standard Markov chain.

In [7]:
mc.Embed()

Warning in Homogeneous1DBirthDeath::Embed(): no structure available for embedding. null result returned.


### Finite-space birth-death proces

A birth-death process with 11 states

In [8]:
mcf = mmc.Homogeneous1DBirthDeath( 11, 0.4, 0.8 )

In [9]:
stadisf = mcf.StationaryDistribution()

In [10]:
print(stadisf)

Discrete distribution values { 0  1  2  3  4  5  6  7  8  9  10  } probas {   0.5002   0.2501   0.1251  0.06253  0.03127  0.01563 0.007816 0.003908 0.001954 0.000977 0.0004885 }


In [11]:
print(stadisf.Mean())
print(stadisf.getProba(0))
print(stadisf.getProba(1))
print(stadisf.Ccdf(4))

0.9946262823644356
0.5002442598925256
0.2501221299462628
0.030776746458231585


Investigating hitting times.

First set up the hitting set. Here, states 3 and 7 are in the set: \[ - - - X - - - X - - - \]

In [12]:
hitSet = np.zeros( [11], dtype=bool)
hitSet[3] = True
hitSet[7] = True
print(hitSet)

[False False False  True False False False  True False False False]


Average hitting times are computed (numerically in this case).

In [13]:
avg = mcf.AverageHittingTime( hitSet )
print(avg)

[27.5        25.         17.5         0.          1.83333333  3.
  2.83333333  0.          2.1875      4.0625      5.3125    ]


Also available, joint expected hitting time and hitting state: E( tau * ind(X(tau)=s) )

In [14]:
avgcon = mcf.AverageHittingTime_Conditional( hitSet )
print( avgcon[0:10,2:8] )

[[ 0.         27.5         0.          0.          0.          0.        ]
 [ 0.         25.          0.          0.          0.          0.        ]
 [ 0.         17.5         0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          1.57777778  0.          0.          0.          0.25555556]
 [ 0.          2.4         0.          0.          0.          0.6       ]
 [ 0.          2.04444444  0.          0.          0.          0.78888889]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          2.1875    ]
 [ 0.          0.          0.          0.          0.          4.0625    ]]


Also available: simulation of hitting times.

In [15]:
simres = mcf.SimulateHittingTime( 0, hitSet, 20, 10000 )

In [16]:
simres.Diagnose()

# Simulation result
# Time type: continuous
# Keeps a trajectory of size 20
# DT trajectory size: 0
# CT trajectory size: 20
# Last state: 0
# Last time:  12.269390


In [17]:
print(simres.CT_dates())

[27.32613419  2.44568247  1.05618749  7.25109476  4.47121101 10.19021112
 92.8707826   6.30537834 16.21055573 38.51771831 55.36265299  5.69199989
  7.63564756 26.88902048  4.51929826 26.23428614  6.88354048 15.6616673
 29.28704501 12.26938997]


### Multidimensional birth-death

Consider a 3-d birth-death process on a 4 x 4 x 4 box.

The constructor to `HomogeneousMultiDBirthDeath` has arguments: sizes, birth rates, death rates.

In [18]:
mdbd = mmc.HomogeneousMultiDBirthDeath( [ 4, 4, 4 ], [ 1.0, 1.0, 1.0], [ 0.8, 0.8, 0.2 ] )

In [19]:
simres = mdbd.SimulateChain( 10.0, stats=True, traj=False, withIncrements=True, _print=True )

[   0]     0.000000        0   0.000000 (   0,   0,   0)
[   1]     1.057601       16   1.057601 (   1,   0,   0)
[   2]     1.407539       20   0.349938 (   1,   1,   0)
[   3]     1.853709       16   0.446170 (   1,   0,   0)
[   4]     2.339213        0   0.485504 (   0,   0,   0)
[   5]     2.528779        4   0.189566 (   0,   1,   0)
[   6]     2.578830        0   0.050051 (   0,   0,   0)
[   7]     3.007722       16   0.428892 (   1,   0,   0)
[   8]     3.337176       17   0.329454 (   1,   0,   1)
[   9]     4.380857       21   1.043680 (   1,   1,   1)
[  10]     5.035321       22   0.654465 (   1,   1,   2)
[  11]     5.055443       23   0.020121 (   1,   1,   3)
[  12]     5.611569       27   0.556127 (   1,   2,   3)
[  13]     5.679364       23   0.067794 (   1,   1,   3)
[  14]     5.841802        7   0.162438 (   0,   1,   3)
[  15]     6.504515        6   0.662713 (   0,   1,   2)
[  16]     6.778746        2   0.274231 (   0,   0,   2)
[  17]     6.796865        3   

0.018119 (   0,   0,   3)
[  18]     7.182389       19   0.385524 (   1,   0,   3)
[  19]     7.454535        3   0.272146 (   0,   0,   3)
[  20]     8.032846       19   0.578311 (   1,   0,   3)
[  21]     8.372880       23   0.340034 (   1,   1,   3)
[  22]     8.455506       27   0.082626 (   1,   2,   3)
[  23]     8.601174       26   0.145668 (   1,   2,   2)
[  24]     8.631567       27   0.030393 (   1,   2,   3)
[  25]     9.080563       43   0.448996 (   2,   2,   3)
[  26]     9.116700       47   0.036137 (   2,   3,   3)
[  27]     9.245999       43   0.129299 (   2,   2,   3)
[  28]     9.648527       47   0.402528 (   2,   3,   3)
[  29]     9.862576       31   0.214049 (   1,   3,   3)
[  30]    10.000000       15   0.137424 (   0,   3,   3)


Statistics can be performed on this trajectory also

In [20]:
sd = simres.Distribution()

# State   0:   cum time =     1.676059   ( 16.7606%)
# State   1:   cum time =     0.000000   (  0.0000%)
# State   2:   cum time =     0.018119   (  0.1812%)
# State   3:   cum time =     0.963835   (  9.6383%)
# State   4:   cum time =     0.050051   (  0.5005%)
# State   5:   cum time =     0.000000   (  0.0000%)
# State   6:   cum time =     0.274231   (  2.7423%)
# State   7:   cum time =     0.662713   (  6.6271%)
# State   8:   cum time =     0.000000   (  0.0000%)
# State   9:   cum time =     0.000000   (  0.0000%)
# State  10:   cum time =     0.000000   (  0.0000%)
# State  11:   cum time =     0.000000   (  0.0000%)
# State  12:   cum time =     0.000000   (  0.0000%)
# State  13:   cum time =     0.000000   (  0.0000%)
# State  14:   cum time =     0.000000   (  0.0000%)
# State  15:   cum time =     0.000000   (  0.0000%)
# State  16:   cum time =     1.164896   ( 11.6490%)
# State  17:   cum time =     1.043680   ( 10.4368%)
# State  18:   cum time =     0.000000   (  0.

000000   (  0.0000%)
# State  61:   cum time =     0.000000   (  0.0000%)
# State  62:   cum time =     0.000000   (  0.0000%)
# State  63:   cum time =     0.000000   (  0.0000%)


## Arrival processes

There are two pure arrival processes in `Marmote`: `PoissonProcess` and `MMPP`.

In [21]:
poi = mmc.PoissonProcess( 1.0 )

In [22]:
simres = poi.SimulateChain( 10.0, False, True, True )

[   0]     0.000000        0   0.000000
[   1]     1.191349        1   1.191349
[   2]     1.601412        2   0.410064
[   3]     1.871647        3   0.270235
[   4]     2.701199        4   0.829552
[   5]     3.433952        5   0.732753
[   6]     3.567857        6   0.133904
[   7]     5.128728        7   1.560871
[   8]     5.624811        8   0.496083
[   9]     6.024166        9   0.399354
[  10]     7.399286       10   1.375120
[  11]     8.297379       11   0.898093
[  12]     8.968802       12   0.671423
[  13]     9.495159       13   0.526357
[  14]    10.000000       14   0.504841


Creating an MMPP with modulating chain the 8-state birth-death of this lesson. Arrival rates are 0 except in states 1 and 7.

In [23]:
mmpp = mmc.MMPP( mcf.generator(), [ 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0 ] )

Simulation stating from 0 arrivals and state 0 of the environment. 

Coding of 'state' trajectory (experimental) is 2*nb of arrivals + current state of the environment.

In [24]:
simres = mmpp.SimulateChain( 10.0, marmotecore.DiracDistribution(0.0), marmotecore.DiracDistribution(0.0), False, True, True )

[   0]     0.000000        0   0.000000
[   1]     5.661661        1   5.661661
[   2]     5.691483        0   0.029821
[   3]     5.888489        1   0.197006
[   4]     6.613527        0   0.725039
[   5]     9.433967        1   2.820439
[   6]     9.774648        0   0.340681
[   7]    10.000000        1   0.225352
